# Parte 3: Generación de musica con Redenes Neuronales Recurrentes (RNNs)

En esta parte del laboratorio, exploraremos la construcción de una red neuronal recurrente (RNN) para la generación de música. Entrenaremos un modelo para aprender los patrones en partituras sin procesar en [notación ABC](https://en.wikipedia.org/wiki/ABC_notation) y luego usaremos este modelo para generar música.



## 3.1 Importar dependencias

In [ ]:
# Import Tensorflow 2.0
%tensorflow_version 2.x
import tensorflow as tf

# Download and import the MIT 6.S191 package
!pip install mitdeeplearning
import mitdeeplearning as mdl

# Import all remaining packages
import numpy as np
import os
import time
import functools
from IPython import display as ipythondisplay
from tqdm import tqdm
!apt-get install abcmidi timidity > /dev/null 2>&1

# Check that we are using a GPU, if not switch runtimes
#   using Runtime > Change Runtime Type > GPU
print("Number of available GPUs:", len(tf.config.list_physical_devices('GPU')))
assert len(tf.config.list_physical_devices('GPU')) > 0


## 3.2 Importar datos

![Let's Dance!](http://33.media.tumblr.com/3d223954ad0a77f4e98a7b87136aa395/tumblr_nlct5lFVbF1qhu7oio1_500.gif)


Se utilizará un conjunto de datos de cientos de canciones populares irlandesas, representadas en la notación ABC.

In [ ]:
# Download the dataset
songs = mdl.lab1.load_training_data()

print("The datasets contains", len(songs), "songs")
# Print one of the songs to inspect it in greater detail!
example_song = songs[42]
print("\nExample song: \n", example_song)


Se convertirá una canción en notación ABC a un audio y se reproducira.

In [ ]:
cont = 1
for i in songs:
    print("Song", cont , "Long:", len(i))
    cont+=1

In [ ]:
# Convert the ABC notation to audio file and listen to it
mdl.lab1.play_song(example_song)

Una cosa importante en la que pensar es que esta notación de música no solo contiene información sobre las notas que se están reproduciendo, sino que además hay metainformación como el título, la clave y el tiempo de la canción. ¿Cómo afecta la cantidad de caracteres diferentes que están presentes en el archivo de texto a la complejidad del problema de aprendizaje? Esto será importante pronto, cuando generemos una representación numérica para los datos de texto.

In [ ]:
# Join our list of song strings into a single string containing all songs
songs_joined = "\n\n".join(songs)
print(songs_joined)

# Find all unique characters in the joined string
vocab = sorted(set(songs_joined))
print("There are", len(vocab), "unique characters in the dataset")
print("Vocab:", vocab)

## 3.3 Procesar el conjunto de datos para la tarea de aprendizaje

Demos un paso atrás y consideremos nuestra tarea de predicción. Estamos tratando de entrenar un modelo RNN para aprender patrones en la música en formato ABC y luego usar este modelo para generar una nueva pieza musical.

Desglosando esto, lo que realmente le estamos preguntando al modelo es: **dado un carácter, o una secuencia de caracteres, ¿cuál es el próximo carácter más probable? Entrenaremos el modelo para realizar esta tarea.**

Para lograr esto, ingresaremos una secuencia de caracteres en el modelo y lo entrenaremos para predecir la salida, es decir, el siguiente carácter en cada paso de tiempo. Las RNNs mantienen un estado interno que depende de los elementos vistos anteriormente, por lo que la información sobre todos los caracteres vistos hasta un momento dado se tendrá en cuenta para generar la predicción.


#### *Vectorizando el texto*

Antes de comenzar a entrenar nuestro modelo de RNN, necesitaremos crear una representación numérica de nuestro conjunto de datos basado en texto. Para hacer esto, generaremos dos tablas de búsqueda: una que asigna caracteres a números y una segunda que asigna números a caracteres. Recuerde que acabamos de identificar los caracteres únicos presentes en el texto.


In [ ]:
### Define numerical representation of text ###

# Create a mapping from character to unique index.
# For example, to get the index of the character "d",
#   we can evaluate `char2idx["d"]`.
char2idx = {u:i for i, u in enumerate(vocab)}

print("Char to int", char2idx)

# Create a mapping from indices to characters. This is
#   the inverse of char2idx and allows us to convert back
#   from unique index to the character in our vocabulary.
idx2char = np.array(vocab)
print("Int to char:", idx2char)

Esto nos da una representación entera para cada carácter. Observe que los caracteres únicos (es decir, nuestro vocabulario) en el texto están mapeados como índices de 0 a len (vocab). Veamos esta representación numérica de nuestro conjunto de datos:

In [ ]:
print('{')
for char,_ in zip(char2idx, range(83)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

In [ ]:
### Vectorize the songs string ###

'''TODO: Write a function to convert the all songs string to a vectorized
    (i.e., numeric) representation. Use the appropriate mapping
    above to convert from vocab characters to the corresponding indices.

  NOTE: the output of the `vectorize_string` function
  should be a np.array with `N` elements, where `N` is
  the number of characters in the input string
'''

def vectorize_string(string):
    #print(string)
    vectorized_output = np.array([char2idx[char] for char in string])
    return vectorized_output

# Call the function
vectorized_songs = vectorize_string(songs_joined)
print("Vectorized songs:", vectorized_songs, len(vectorized_songs))

También podemos ver cómo se asigna la primera parte del texto a una representación entera:

In [ ]:
print ('{} ---- characters mapped to int ----> {}'.format(repr(songs_joined[:10]), vectorized_songs[:10]))
# check that vectorized_songs is a numpy array
assert isinstance(vectorized_songs, np.ndarray), "returned result should be a numpy array"

### Crea ejemplos para el entrnamiento

Nuestro siguiente paso es dividir el texto en secuencias de ejemplo que usaremos durante el entrenamiento. Cada secuencia de entrada que alimentamos a nuestra RNN contendrá caracteres `seq_length` del texto. También necesitaremos definir una secuencia objetivo para cada secuencia de entrada, que se utilizará en el entrenamiento de la RNN para predecir el siguiente carácter. Para cada entrada, el destino correspondiente contendrá la misma longitud de texto, excepto que se desplazará un carácter hacia la derecha.

Para hacer esto, dividiremos el texto en trozos de `seq_length + 1`. Suponga que `seq_length` es 4 y nuestro texto es" Hola ". Entonces, nuestra secuencia de entrada es "Hol" y la secuencia objetivo es "ola".

El método por lotes nos permitirá convertir este flujo de índices de caracteres en secuencias del tamaño deseado.

In [ ]:
### Batch definition to create training examples ###

def get_batch(vectorized_songs, seq_length, batch_size):
  # the length of the vectorized songs string
  n = vectorized_songs.shape[0] - 1
  #print("n:", n)
  # randomly choose the starting indices for the examples in the training batch

  #print("Random choice:",n-seq_length, batch_size)
  idx = np.random.choice(n-seq_length, batch_size) # Two samples
  #print("idx:", idx)

  '''TODO: construct a list of input sequences for the training batch'''
  input_batch = [vectorized_songs[i : i+seq_length] for i in idx]
  # input_batch = # TODO
  #print("Input_batch:", input_batch)
  '''TODO: construct a list of output sequences for the training batch'''
  output_batch = [vectorized_songs[i+1 : i+seq_length+1] for i in idx]
  #print("Output_batch:", output_batch)
  #print()
  # output_batch = # TODO

  # x_batch, y_batch provide the true inputs and targets for network training
  x_batch = np.reshape(input_batch, [batch_size, seq_length])
  y_batch = np.reshape(output_batch, [batch_size, seq_length])
  return x_batch, y_batch


# Perform some simple tests to make sure your batch function is working properly!
test_args = (vectorized_songs, 10, 2)

if not mdl.lab1.test_batch_func_types(get_batch, test_args) or \
   not mdl.lab1.test_batch_func_shapes(get_batch, test_args) or \
   not mdl.lab1.test_batch_func_next_step(get_batch, test_args):
   print("======\n[FAIL] could not pass tests")
else:
   print("======\n[PASS] passed all tests!")

Para cada uno de estos vectores, cada índice se procesa en un solo paso de tiempo. Entonces, para la entrada en el paso de tiempo 0, el modelo recibe el índice del primer carácter de la secuencia e intenta predecir el índice del siguiente carácter. En el siguiente paso de tiempo, hace lo mismo, pero la RNN considera la información del paso anterior, es decir, su estado actualizado, además de la entrada actual.

Podemos hacer esto al observar cómo funciona con los primeros caracteres de nuestro texto:

In [ ]:
x_batch, y_batch = get_batch(vectorized_songs, seq_length=5, batch_size=1)
#print("x_batch:", x_batch)
#print("y_batch:", y_batch)


for i, (input_idx, target_idx) in enumerate(zip(np.squeeze(x_batch), np.squeeze(y_batch))):
    print("Step {:3d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[int(input_idx)])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[int(target_idx)])))

## 3.4 El modelo de red neuronal recurrente (RNN)

Ahora estamos listos para definir y entrenar un modelo RNN utilizando un conjunto de datos de música en formato ABC, y luego usar ese modelo entrenado para generar una nueva canción. Entrenaremos a nuestra RNN usando lotes de fragmentos de canciones de nuestro conjunto de datos, que generamos en la sección anterior.

El modelo se basa en la arquitectura LSTM, donde usamos un vector de estados para mantener información sobre las relaciones temporales entre caracteres consecutivos. La salida final del LSTM se alimenta a una capa [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense) completamente conectada donde utilizarémos una función softmax sobre cada carácter del vocabulario, y luego muestrear de esta distribución para predecir el próximo carácter.

Como presentamos en la primera parte de esta unidad, usaremos la API de Keras, específicamente, [`tf.keras.Sequential`](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential). Se utilizan tres capas para definir el modelo:

* [`tf.keras.layers.Embedding`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding): esta es la capa de entrada, que consta de una tabla de búsqueda entrenable que asigna los números de cada carácter a un vector con dimensiones "embedding_dim".
* [`tf.keras.layers.LSTM`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM): Nuestra red LSTM, con tamaño` units = rnn_units`.
* [`tf.keras.layers.Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense): La capa de salida, con salidas`vocab_size`.

<img src="https://raw.githubusercontent.com/aamini/introtodeeplearning/2019/lab1/img/lstm_unrolled-01-01.png" alt="Drawing"/>

### Definir el modelo RNN

Ahora, definiremos una función que usaremos para construir el modelo.

In [ ]:
def LSTM(rnn_units):
  return tf.keras.layers.LSTM(
    rnn_units,
    return_sequences=True,
    recurrent_initializer='glorot_uniform',
    recurrent_activation='sigmoid',
    stateful=True,
  )

¡El tiempo ha llegado! Complete los `TODOs` para definir el modelo RNN dentro de la función` build_model`, y luego llame a la función que acaba de definir para crear una instancia del modelo.

In [ ]:
### Defining the RNN Model ###

'''TODO: Add LSTM and Dense layers to define the RNN model using the Sequential API.'''
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    # Layer 1: Embedding layer to transform indices into dense vectors
    #   of a fixed embedding size
    tf.keras.layers.Embedding(vocab_size, embedding_dim),

    # Layer 2: LSTM with `rnn_units` number of units.
    # TODO: Call the LSTM function defined above to add this layer.
    LSTM(rnn_units),
    # LSTM('''TODO'''),

    # Layer 3: Dense (fully-connected) layer that transforms the LSTM output
    #   into the vocabulary size.
    # TODO: Add the Dense layer.
    tf.keras.layers.Dense(vocab_size)
    # '''TODO: DENSE LAYER HERE'''
  ])

  return model

# Build a simple model with default hyperparameters. You will get the
#   chance to change these later.
model = build_model(len(vocab), embedding_dim=256, rnn_units=1024, batch_size=32)
model.summary()

### Prueba el modelo RNN

Siempre es una buena idea ejecutar algunas comprobaciones simples en nuestro modelo para ver si se comporta como se esperaba.

Primero, podemos usar la función `Model.summary` para imprimir un resumen del funcionamiento interno de nuestro modelo. Aquí podemos comprobar las capas en el modelo, la forma de la salida de cada una de las capas, el tamaño del lote, etc.

In [ ]:
model.summary()

También podemos comprobar rápidamente la dimensionalidad de nuestra salida, utilizando una longitud de secuencia de 100. Tenga en cuenta que el modelo se puede ejecutar en entradas de cualquier longitud.

In [ ]:
x, y = get_batch(vectorized_songs, seq_length=100, batch_size=32)
pred = model(x)
#print("Input shape:      ", x.shape, " # (batch_size, sequence_length)")
#print("Prediction shape: ", pred.shape, "# (batch_size, sequence_length, vocab_size)")

### Predicciones del modelo no entrenado

Echemos un vistazo a lo que predice nuestro modelo no entrenado.

Para obtener predicciones reales del modelo, tomamos muestras de la distribución de salida, que está definida por una función de activación "softmax" sobre nuestro vocabulario de caracteres. Esto nos dará índices de caracteres reales. Esto significa que estamos usando una [distribución categórica](https://en.wikipedia.org/wiki/Categorical_distribution) para muestrear la predicción de ejemplo. Esto da una predicción del siguiente carácter (específicamente su índice) en cada paso de tiempo.

Tenga en cuenta que tomamos muestras de esta distribución de probabilidad, en lugar de simplemente tomar el `argmax`, lo que puede hacer que el modelo se cicle.

Probemos este muestreo para el primer ejemplo del lote.

In [ ]:
sampled_indices = tf.random.categorical(pred[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

Ahora podemos decodificarlos para ver el texto predicho por el modelo no entrenado:

In [ ]:
print("Input: \n", repr("".join(idx2char[int(x[0,i])] for i in range(len(x[0])))))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices])))

Como puedes ver, el texto predicho por el modelo no entrenado no tiene sentido. ¿Cómo podemos hacerlo mejor? ¡Podemos entrenar la red!

## 3.5 Entrenamiento del modelo: operaciones de pérdida y entrenamiento

¡Ahora es el momento de entrenar al modelo!

En este punto, podemos pensar en nuestro próximo problema de predicción de caracteres como un problema de clasificación estándar. Dado el estado anterior de la RNN, así como la entrada en un intervalo de tiempo determinado, queremos predecir la clase del siguiente carácter, es decir, predecir realmente el próximo carácter.

Para entrenar nuestro modelo en esta tarea de clasificación, podemos usar una forma de pérdida de "entropía cruzada" (pérdida de probabilidad logarítmica negativa). Específicamente, usaremos como función de pérdida: [`sparse_categorical_crossentropy`](https://www.tensorflow.org/api_docs/python/tf/keras/losses/sparse_categorical_crossentropy), ya que utiliza objetivos enteros para tareas de clasificación categórica. Querremos calcular la pérdida utilizando los verdaderos objetivos, las "etiquetas", y los objetivos predichos, los "logis".

Primero calculemos la pérdida usando nuestras predicciones de ejemplo del modelo no entrenado:

In [ ]:
### Defining the loss function ###

'''TODO: define the loss function to compute and return the loss between
    the true labels and predictions (logits). Set the argument from_logits=True.'''
def compute_loss(labels, logits):
  loss = tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
  # loss = tf.keras.losses.sparse_categorical_crossentropy('''TODO''', '''TODO''', from_logits=True) # TODO
  return loss

'''TODO: compute the loss using the true next characters from the example batch
    and the predictions from the untrained model several cells above'''
example_batch_loss = compute_loss(y, pred)
# example_batch_loss = compute_loss('''TODO''', '''TODO''') # TODO

print("Prediction shape: ", pred.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Comencemos definiendo algunos **hiperparámetros** para entrenar el modelo. Para empezar, hemos proporcionado algunos valores razonables para algunos de los parámetros. ¡Depende de usted usar lo que hemos aprendido en clase para ayudar a optimizar la selección de parámetros aquí!

In [ ]:
import os
### Hyperparameter setting and optimization ###

# Optimization parameters:
num_training_iterations = 500  # Increase this to train longer
batch_size = 32  # Experiment between 1 and 64
seq_length = 100  # Experiment between 50 and 500
learning_rate = 3e-3  # Experiment between 1e-5 and 1e-1

# Model parameters:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024  # Experiment between 1 and 2048

# Checkpoint location:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "my_ckpt")

Ahora, estamos listos para definir nuestra operación de entrenamiento - el optimizador y la duración del entrenamiento - y usar esta función para entrenar el modelo. Experimentaremos con la elección del optimizador y el número de iteraciones (epochs), y verás cómo estos cambios afectan la salida de la red. Algunos optimizadores a probar son [`Adam`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam?version=stable) y [` Adagrad`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adagrad?version=stable).

Primero, crearemos una instancia de un nuevo modelo y un optimizador. Luego, usaremos el método [`tf.GradientTape`](https://www.tensorflow.org/api_docs/python/tf/GradientTape) para realizar las operaciones de `"back propagation"`.

También generaremos una impresión del progreso del modelo a través del entrenamiento, que nos ayudará a visualizar fácilmente si estamos minimizando o no la pérdida.

In [ ]:
### Define optimizer and training operation ###

'''TODO: instantiate a new model for training using the `build_model`
  function and the hyperparameters created above.'''
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)
# model = build_model('''TODO: arguments''')

'''TODO: instantiate an optimizer with its learning rate.
  Checkout the tensorflow website for a list of supported optimizers.
  https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/
  Try using the Adam optimizer to start.'''
optimizer = tf.keras.optimizers.Adagrad(learning_rate)
# optimizer = # TODO

@tf.function
def train_step(x, y):
  # Use tf.GradientTape()
  with tf.GradientTape() as tape:

    '''TODO: feed the current input into the model and generate predictions'''
    y_hat = model(x) # TODO
    # y_hat = model('''TODO''')

    '''TODO: compute the loss!'''
    loss = compute_loss(y, y_hat) # TODO
    # loss = compute_loss('''TODO''', '''TODO''')

  # Now, compute the gradients
  '''TODO: complete the function call for gradient computation.
      Remember that we want the gradient of the loss with respect all
      of the model parameters.
      HINT: use `model.trainable_variables` to get a list of all model
      parameters.'''
  grads = tape.gradient(loss, model.trainable_variables) # TODO
  # grads = tape.gradient('''TODO''', '''TODO''')

  # Apply the gradients to the optimizer so it can update the model accordingly
  optimizer.apply_gradients(zip(grads, model.trainable_variables))
  return loss

##################
# Begin training!#
##################

history = []
plotter = mdl.util.PeriodicPlotter(sec=0.5, xlabel='Iterations', ylabel='Loss')
if hasattr(tqdm, '_instances'): tqdm._instances.clear() # clear if it exists

for iter in tqdm(range(num_training_iterations)): #tqdm displays progress

  # Grab a batch and propagate it through the network
  x_batch, y_batch = get_batch(vectorized_songs, seq_length, batch_size)
  loss = train_step(x_batch, y_batch)

  # Update the progress bar
  history.append(loss.numpy().mean())
  plotter.plot(history)

  # Update the model with the changed weights!
  #if iter % 100 == 0:
  #model.save_weights(checkpoint_prefix+str(".weights.h5"))

# Save the trained model and the weights
model.save_weights(checkpoint_prefix+".weights.h5")


## 3.6 Generar música usando el modelo RNN

¡Ahora, podemos usar nuestro modelo RNN entrenado para generar algo de música! Al generar música, tendremos que alimentar el modelo con algún tipo de datos iniciales para que comience (¡porque no puede predecir nada sin algo para comenzar!).

Una vez que tenemos datos iniciales (semilla generada), podemos predecir iterativamente cada carácter sucesivo (recuerda, estamos usando la representación ABC para nuestra música) usando nuestra RNN entrenado. Específicamente, recuerda que nuestra RNN utiliza una función de activación "softmax" sobre posibles caracteres sucesivos. Para la inferencia, tomamos muestras iterativas de estas distribuciones y luego usamos nuestras muestras para codificar una canción generada en el formato ABC.

Entonces, ¡todo lo que tenemos que hacer es escribirlo en un archivo y escuchar!

### Restaurar el último punto de control

Para mantener este paso de inferencia simple, usaremos un tamaño de lote de 1. Debido a cómo se pasa el estado RNN de un paso de tiempo a otro, el modelo solo podrá aceptar un tamaño de lote fijo una vez que esté construido.

Para ejecutar el modelo con un `batch_size` diferente, necesitaremos reconstruir, volver a entrenar el modelo y restaurar los pesos desde el último punto de control, es decir, los pesos después del último punto de control durante el entrenamiento:

In [ ]:
'''TODO: Rebuild the model using a batch_size=1'''
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1) # TODO
# model = build_model('''TODO''', '''TODO''', '''TODO''', batch_size=1)

# Restore the model weights for the last checkpoint after training
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

model.summary()

Observe que hemos introducido un `batch_size` fijo de 1 para la inferencia.

### El procedimiento de predicción

Ahora, estamos listos para escribir el código para generar texto en el formato de música ABC:

* Inicializar una cadena de inicio "semilla" y el estado RNN, y establecer el número de caracteres que queremos generar.

* Utilice la cadena de inicio y el estado RNN para obtener la distribución de probabilidad sobre el siguiente carácter predicho.

* Muestra de distribución multinomial para calcular el índice del carácter predicho. Este carácter predicho se utiliza luego como la siguiente entrada al modelo.

* En cada paso de tiempo, el estado de la RNN actualizado se retroalimenta en el modelo, de modo que ahora tiene más contexto para hacer la próxima predicción. Después de predecir el siguiente carácter, los estados RNN actualizados se retroalimentan nuevamente en el modelo, que es la forma en que aprende las dependencias de secuencia en los datos, a medida que obtiene más información de las predicciones anteriores.

![Inferencia de LSTM](https://raw.githubusercontent.com/aamini/introtodeeplearning/2019/lab1/img/lstm_inference.png)

Complete y experimente con este bloque de código (¡así como con algunos de los aspectos de la definición y el entrenamiento de la red!), Y vea cómo funciona el modelo. ¿Cómo se comparan las canciones generadas después del entrenamiento con un pequeño número de épocas con las generadas después de un entrenamiento de mayor duración?

In [ ]:
### Prediction of a generated song ###

def generate_text(model, start_string, generation_length=1000):
  # Evaluation step (generating ABC text using the learned RNN model)

  '''TODO: convert the start string to numbers (vectorize)'''
  input_eval = [char2idx[s] for s in start_string] # TODO
  # input_eval = ['''TODO''']
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Here batch size == 1
  model.reset_states()
  tqdm._instances.clear()

  for i in tqdm(range(generation_length)):
      '''TODO: evaluate the inputs and generate the next character predictions'''
      predictions = model(input_eval)
      # predictions = model('''TODO''')

      # Remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      '''TODO: use a multinomial distribution to sample'''
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      # predicted_id = tf.random.categorical('''TODO''', num_samples=1)[-1,0].numpy()

      # Pass the prediction along with the previous hidden state
      #   as the next inputs to the model
      input_eval = tf.expand_dims([predicted_id], 0)

      '''TODO: add the predicted character to the generated text!'''
      # Hint: consider what format the prediction is in vs. the output
      text_generated.append(idx2char[predicted_id]) # TODO
      # text_generated.append('''TODO''')

  return (start_string + ''.join(text_generated))

In [ ]:
'''TODO: Use the model and the function defined above to generate ABC format text of length 1000!
    As you may notice, ABC files start with "X" - this may be a good start string.'''
generated_text = generate_text(model, start_string="X", generation_length=3000) # TODO
print("Gen text:", generated_text)
# generated_text = generate_text('''TODO''', start_string="X", generation_length=1000)

### ¡Reproduce la música generada!

¡Ahora podemos llamar a una función para convertir el texto en formato ABC en un archivo de audio, y luego reproducirlo para ver nuestra música generada! Intente entrenar más tiempo si la canción resultante no es lo suficientemente larga, ¡o vuelva a generar la canción!

In [ ]:
### Play back generated songs ###

generated_songs = mdl.lab1.extract_song_snippet(generated_text)

for i, song in enumerate(generated_songs):
  # Synthesize the waveform from a song
  waveform = mdl.lab1.play_song(song)

  # If its a valid song (correct syntax), lets play it!
  if waveform:
    print("Generated song", i)
    ipythondisplay.display(waveform)

## 3.7 ¡Experimenta y produce tus propias canciones!

¡Felicitaciones por hacer su primer modelo de secuencia en TensorFlow! Es un logro bastante grande, y espero que tengas algunas melodías dulces para mostrar.

Considere cómo puede mejorar su modelo y qué parece ser más importante en términos de rendimiento. Aquí hay algunas ideas para comenzar:

* ¿Cómo afecta el número de épocas de entrenamiento al rendimiento?
* ¿Qué sucede si modifica o aumenta el conjunto de datos?
* ¿La elección de la cadena de inicio afecta significativamente el resultado?

## 3.8 Ejercicio

+ Generar automáticamente otro tipo de musica.

> Ejemplo: [Mathematical Mozart GitHub](https://github.com/sarthakagarwal18/Mathematical-Mozart)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# 1. Descargar datos de Google usando yfinance
df = yf.download('GOOGL', start='2010-01-01', end='2022-01-01')
df = df[['Close']]

# 2. Preprocesar los datos
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df.values)

train_size = int(len(df) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

# Crear datos de entrenamiento
def create_dataset(data, time_step=60):
    x, y = [], []
    for i in range(len(data) - time_step):
        x.append(data[i:i + time_step])
        y.append(data[i + time_step])
    return np.array(x), np.array(y)

time_step = 60
x_train, y_train = create_dataset(train_data, time_step)
x_test, y_test = create_dataset(test_data, time_step)

# Reshape para LSTM (samples, time_steps, features)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# 3. Crear modelo LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=25))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')

# 4. Entrenar el modelo
model.fit(x_train, y_train, batch_size=64, epochs=10)

# 5. Hacer predicciones y evaluar el modelo
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

# Visualización
train = df[:train_size]
valid = df[train_size:]
valid['Predictions'] = predictions

plt.figure(figsize=(16, 8))
plt.title('Predicción de precios de acciones de Google')
plt.plot(train['Close'], label='Datos de entrenamiento')
plt.plot(valid['Close'], label='Real')
plt.plot(valid['Predictions'], label='Predicciones')
plt.legend(['Entrenamiento', 'Real', 'Predicciones'], loc='lower right')
plt.show()
